Credit Card Recommender


In [9]:
import gradio as gr
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime, timedelta
import tempfile
import os

## Card Database with Matching Rules

In [10]:
# Each card has categories - if user's spending matches, the card matches user inputs 
DBS_CARDS = {
    "esso": {
        "name": "DBS Esso Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/dbs-esso-card",
        "min_income": 30000,
        "type": "cashback",
        "priority_match": ["petrol"],  # If user selects petrol, this card wins
        "benefit": "21.8% savings on Esso petrol",
        "description": "Best for drivers who spend on petrol regularly"
    },
    "altitude": {
        "name": "DBS Altitude Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/dbs-altitude-visa-signature-card",
        "min_income": 30000,
        "type": "miles",
        "priority_match": ["travel"],  # If user selects travel + wants miles
        "benefit": "3 miles per $1 overseas, 2 miles on travel bookings",
        "description": "Best for frequent travelers who want to earn miles"
    },
    "yuu": {
        "name": "DBS yuu Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/dbs-yuu-cards",
        "min_income": 30000,
        "type": "cashback",
        "priority_match": ["groceries"],  # If user selects groceries
        "benefit": "10 yuu points per $1 at Cold Storage, Giant, Guardian",
        "description": "Best for families who shop at Cold Storage, Giant, Guardian"
    },
    "live_fresh": {
        "name": "DBS Live Fresh Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/live-fresh-card",
        "min_income": 30000,
        "type": "cashback",
        "priority_match": ["online shopping"],  # If user selects online
        "benefit": "5% cashback on online spending and Grab",
        "description": "Best for young professionals who shop online and use Grab"
    },
    "everyday": {
        "name": "POSB Everyday Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/posb-everyday-card",
        "min_income": 30000,
        "type": "cashback",
        "priority_match": ["dining"],  # If user selects dining
        "benefit": "5% cashback on dining, groceries, and transport",
        "description": "Best for balanced everyday spending on dining, groceries, transport"
    },
    "womans_world": {
        "name": "DBS Womans World Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/dbs-woman-world-card",
        "min_income": 80000,
        "type": "miles",
        "priority_match": [],  # High income + miles preference
        "benefit": "4 miles per $1 on online spending",
        "description": "Premium card for high earners who want miles from online spending"
    },
    "vantage": {
        "name": "DBS Vantage Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/dbs-vantage-card",
        "min_income": 120000,
        "type": "miles",
        "priority_match": [],  # Very high income + miles
        "benefit": "Up to 6 miles per $1, premium lounge access",
        "description": "Ultra-premium card for high earners who travel frequently"
    },
    "takashimaya": {
        "name": "DBS Takashimaya Card",
        "url": "https://www.dbs.com.sg/personal/cards/credit-cards/dbs-takashimaya-cards",
        "min_income": 30000,
        "type": "cashback",
        "priority_match": ["shopping"],  # If user selects retail shopping
        "benefit": "5-8% rebate at Takashimaya",
        "description": "Best for frequent Takashimaya shoppers"
    }
}

## Card Matcher with spending categories 

In [11]:
def match_card(spending_categories: List[str], wants_miles: bool, income: float) -> Dict:
    """
    Card matching based on below spending habits 
    
    Priority order:
    Petrol → Esso Card
    Travel + Miles → Altitude Card (or Vantage if high income)
    Groceries → yuu Card
    Online Shopping → Live Fresh Card
    Dining → Everyday Card
    Shopping → Takashimaya Card
    Default: Live Fresh (cashback) or Altitude (miles)
    """
    
    spending_lower = [s.lower() for s in spending_categories]
    
    # Rule 1: PETROL - highest priority for drivers
    if "petrol" in spending_lower:
        print("   ✓ Match: PETROL → Esso Card")
        return DBS_CARDS["esso"]
    
    # Rule 2: TRAVEL + wants miles
    if "travel" in spending_lower and wants_miles:
        if income >= 120000:
            print("   ✓ Match: TRAVEL + HIGH INCOME → Vantage Card")
            return DBS_CARDS["vantage"]
        else:
            print("   ✓ Match: TRAVEL + MILES → Altitude Card")
            return DBS_CARDS["altitude"]
    
    # Rule 3: GROCERIES
    if "groceries" in spending_lower:
        print("   ✓ Match: GROCERIES → yuu Card")
        return DBS_CARDS["yuu"]
    
    # Rule 4: ONLINE SHOPPING
    if "online shopping" in spending_lower or "online" in spending_lower:
        if wants_miles and income >= 80000:
            print("   ✓ Match: ONLINE + MILES + HIGH INCOME → Womans World Card")
            return DBS_CARDS["womans_world"]
        else:
            print("   ✓ Match: ONLINE → Live Fresh Card")
            return DBS_CARDS["live_fresh"]
    
    # Rule 5: DINING
    if "dining" in spending_lower:
        print("   ✓ Match: DINING → Everyday Card")
        return DBS_CARDS["everyday"]
    
    # Rule 6: SHOPPING (retail)
    if "shopping" in spending_lower or "retail" in spending_lower:
        print("   ✓ Match: SHOPPING → Takashimaya Card")
        return DBS_CARDS["takashimaya"]
    
    # Rule 7: DEFAULT based on preference
    if wants_miles:
        if income >= 120000:
            print("   ✓ Default: MILES + HIGH INCOME → Vantage Card")
            return DBS_CARDS["vantage"]
        elif income >= 80000:
            print("   ✓ Default: MILES + GOOD INCOME → Womans World Card")
            return DBS_CARDS["womans_world"]
        else:
            print("   ✓ Default: MILES → Altitude Card")
            return DBS_CARDS["altitude"]
    else:
        print("   ✓ Default: CASHBACK → Live Fresh Card")
        return DBS_CARDS["live_fresh"]

## Calendar Generator

In [12]:
def generate_calendar_reminder(card_name: str, customer_name: str, days_from_now: int = 30) -> str:
    reminder_date = datetime.now() + timedelta(days=days_from_now)
    date_str = reminder_date.strftime("%Y%m%d")
    created_str = datetime.now().strftime("%Y%m%dT%H%M%SZ")
    uid = f"dbs-{datetime.now().strftime('%Y%m%d%H%M%S')}@app"
    
    ics = f"""BEGIN:VCALENDAR
VERSION:2.0
PRODID:-//DBS Recommender//EN
BEGIN:VEVENT
UID:{uid}
DTSTART;VALUE=DATE:{date_str}
SUMMARY:Check {card_name} Application
DESCRIPTION:Check your {card_name} application status. DBS Hotline: 1800 111 1111
END:VEVENT
END:VCALENDAR"""
    
    filepath = os.path.join(tempfile.gettempdir(), f"dbs_reminder_{date_str}.ics")
    with open(filepath, 'w') as f:
        f.write(ics)
    return filepath

## Customer Profile & Agent

In [13]:
MODEL_NAME = "llama3.2"
KNOWLEDGE_BASE_PATH = "dbs_credit_card_knowledge_base_FT.txt"
CHROMA_DIR = "./chroma_dbs_rules"

@dataclass
class CustomerProfile:
    name: str
    income: float
    age: int
    spending: List[str]
    monthly_spend: float
    wants_miles: bool


class RuleBasedAgent:
    """
    Agent that:
    1. Uses RULES to pick the card (deterministic)
    2. Uses LLM to EXPLAIN why (natural language)
    """
    
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
    
    def run(self, customer: CustomerProfile) -> Dict:
        # Step 1: RULE-BASED card selection 
        matched_card = match_card(
            customer.spending, 
            customer.wants_miles, 
            customer.income
        )
        
        # Step 2: Get KB info for context
        docs = self.retriever.invoke(matched_card["name"])
        kb_context = "\n".join([d.page_content for d in docs[:3]])
        
        # Step 3: LLM generates explanation
        prompt = f"""You are a credit card advisor. The system has already selected the best card for this customer.
Your job is to EXPLAIN why this card is perfect for them.

CUSTOMER:
- Name: {customer.name}
- Income: S${customer.income:,.0f}/year
- Age: {customer.age}
- Top Spending: {', '.join(customer.spending)}
- Monthly Spend: S${customer.monthly_spend:,.0f}
- Wants: {'Miles' if customer.wants_miles else 'Cashback'}

SELECTED CARD: {matched_card['name']}
KEY BENEFIT: {matched_card['benefit']}
CARD TYPE: {matched_card['type'].upper()}

ADDITIONAL INFO:
{kb_context[:1000]}

Write a personalized explanation (3-4 sentences) for why {matched_card['name']} is the best choice for {customer.name}.
Include a specific reward estimate based on their S${customer.monthly_spend:,.0f} monthly spend.

Explanation:"""
        
        explanation = self.llm.invoke(prompt)
        
        return {
            "card": matched_card,
            "explanation": explanation
        }

## Initialize

In [14]:
llm = OllamaLLM(model=MODEL_NAME, temperature=0.3)
embeddings = OllamaEmbeddings(model=MODEL_NAME)
print(f"✅ LLM: {MODEL_NAME}")

with open(KNOWLEDGE_BASE_PATH, 'r', encoding='utf-8') as f:
    kb = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = [Document(page_content=c) for c in splitter.split_text(kb)]

vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=CHROMA_DIR)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

agent = RuleBasedAgent(llm, retriever)
print("✅ Agent ready!")

✅ LLM: llama3.2
✅ Agent ready!


## Gradio Interface

In [17]:
last_result = {"card_name": "", "customer_name": ""}

def recommend(
    name, age, income, monthly_spend,
    sp_dining, sp_travel, sp_online, sp_groceries, sp_petrol, sp_shopping,
    reward_pref
):
    global last_result
    
    # Build spending list
    spending = []
    if sp_dining: spending.append("dining")
    if sp_travel: spending.append("travel")
    if sp_online: spending.append("online shopping")
    if sp_groceries: spending.append("groceries")
    if sp_petrol: spending.append("petrol")
    if sp_shopping: spending.append("shopping")
    if not spending: spending = ["general"]
    
    customer = CustomerProfile(
        name=name or "Customer",
        income=income,
        age=age,
        spending=spending,
        monthly_spend=monthly_spend,
        wants_miles=(reward_pref == "Miles")
    )
    
    # Profile summary
    profile = f"""
👤 {customer.name}
💰 Income: S${customer.income:,.0f}/year
🎂 Age: {customer.age}
🛒 Spending: {', '.join(customer.spending)}
💳 Monthly: S${customer.monthly_spend:,.0f}
🎁 Wants: {'Miles' if customer.wants_miles else 'Cashback'}
"""
    
    try:
        result = agent.run(customer)
        card = result["card"]
        
        last_result["card_name"] = card["name"]
        last_result["customer_name"] = customer.name
        
        # Format output
        rec_text = f"""## 💳 {card['name']}

Why this card? {card['description']}

Key Benefit: {card['benefit']}


Personalized Analysis

{result['explanation']}


🔗 Apply Now

[👉 Click here to apply for {card['name']}]({card['url']})
"""        
    except Exception as e:
        import traceback
        traceback.print_exc()
        rec_text = f"Error: {e}"
    
    return profile, rec_text
 

def download_calendar():
    if last_result["card_name"]:
        return generate_calendar_reminder(last_result["card_name"], last_result["customer_name"])
    return None

In [18]:
with gr.Blocks(title="Card Recommender") as demo:
    gr.Markdown("Credit Card Recommender\n\n*Different spending = Different card recommendations!*\n\n---")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("Your Profile")
            name = gr.Textbox(label="Name", value="Customer")
            age = gr.Slider(label="Age", minimum=18, maximum=70, value=30)
            income = gr.Slider(label="Annual Income (S$)", minimum=20000, maximum=300000, value=60000, step=5000)
            monthly = gr.Slider(label="Monthly Spend (S$)", minimum=500, maximum=20000, value=3000, step=500)
            
            gr.Markdown("Your TOP Spending Category")
            gr.Markdown("*Select your PRIMARY spending - this determines your card!*")
            with gr.Row():
                sp_petrol = gr.Checkbox(label="⛽ Petrol")
                sp_travel = gr.Checkbox(label="✈️ Travel")
                sp_groceries = gr.Checkbox(label="🥬 Groceries")
            with gr.Row():
                sp_online = gr.Checkbox(label="🛒 Online")
                sp_dining = gr.Checkbox(label="🍽️ Dining")
                sp_shopping = gr.Checkbox(label="🛍️ Shopping")
            
            reward = gr.Radio(["Cashback", "Miles"], label="Reward Type", value="Cashback")
            btn = gr.Button("Get Recommendation", variant="primary", size="lg")
        
        with gr.Column():
            gr.Markdown("Results")
            profile_out = gr.Textbox(label="Your Profile", lines=7)
            rec_out = gr.Markdown()
            
            cal_btn = gr.Button("📅 Download Calendar Reminder")
            cal_file = gr.File(label="Reminder")
    
    gr.Markdown("---\n Try Different Spending Categories")
    gr.Examples(
        [
            ["Milo", 40, 60000, 3000, False, False, False, False, True, False, "Cashback"],   # Petrol → Esso
            ["Kopi", 35, 80000, 4000, False, True, False, False, False, False, "Miles"],       # Travel → Altitude
            ["Teh", 38, 55000, 2500, False, False, True, False, False, False, "Cashback"], # Groceries → yuu
            ["Kaya", 28, 45000, 2000, False, False, False, True, False, False, "Cashback"],# Online → Live Fresh
            ["Toast", 32, 50000, 2200, True, False, False, False, False, False, "Cashback"], # Dining → Everyday
        ],
        inputs=[name, age, income, monthly, sp_dining, sp_travel, sp_groceries, sp_online, sp_petrol, sp_shopping, reward]
    )
    
    btn.click(recommend, 
              [name, age, income, monthly, sp_dining, sp_travel, sp_online, sp_groceries, sp_petrol, sp_shopping, reward],
              [profile_out, rec_out])
    cal_btn.click(download_calendar, [], [cal_file])

In [19]:
demo.launch(share=True, theme=gr.themes.Soft())

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d88ee95aa8dde84d1f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
